# Computed Tomography Simulator

In [1]:
import numpy as np
from matplotlib import pyplot as plt
#import skimage import io
from skimage.util import img_as_int
import io
#import cv2
from PIL import Image, ImageOps
from bresenham import bresenham
import ipywidgets as widgets
import pydicom
from pydicom.data import get_testdata_file
from pydicom.filebase import DicomBytesIO
from IPython.display import clear_output

In [2]:
def find_emiter_position(angle, r, center_x, center_y):
    a = np.deg2rad(angle)
    xe = int(round(r * np.cos(a)) + center_x)
    ye = int(round(r * np.sin(a)) + center_y)

    return [xe, ye]


def find_detector_positions(n, angle, r, l, center_x, center_y):
    a = np.deg2rad(angle)
    rad_l = np.deg2rad(l)

    positions = []
    for i in range(n):
        xd = int(round(r * np.cos(a + np.pi - rad_l / 2 + i * rad_l / (n - 1))) + center_x)
        yd = int(round(r * np.sin(a + np.pi - rad_l / 2 + i * rad_l / (n - 1))) + center_y)
        positions.append([xd, yd])

    return positions


def radon_transform(image_in, alpha, n, l, use_filtering, use_normalize, is_animated):
    size_x = image_in.shape[1]
    size_y = image_in.shape[0]
    center_x = int(size_x / 2)
    center_y = int(size_y / 2)
    d = (size_x ** 2 + size_y ** 2) ** 0.5
    r = d / 2

    pixels = []
    iterations = int(360. / alpha)
    angles = np.linspace(0., 360., iterations, endpoint=False)

    animation_images = []
    animation_pixels = np.zeros((iterations, n))

    for i, angle in enumerate(angles):
        emiter = find_emiter_position(angle, r, center_x, center_y)
        detectors = find_detector_positions(n, angle, r, l, center_x, center_y)

        values = []
        for detector in detectors:
            value_of_pixels = 0
            num_of_pixels = 0
            line = bresenham(emiter[0], emiter[1], detector[0], detector[1])

            for pixel in line:
                px, py = pixel[0], pixel[1]
                if (px >= 0) and (py >= 0) and (px < size_x) and (py < size_y):
                    num_of_pixels += 1
                    value_of_pixels += image_in[py, px]

            if num_of_pixels > 0:
                values.append(np.float64(value_of_pixels / num_of_pixels))
            else:
                values.append(np.float64(0))

        pixels.append(values)

        if is_animated:
            animation_pixels[i] = values
            anisino = animation_pixels.copy()
            if use_normalize:
                anisino = normalize(anisino)
            anisino = np.array(anisino).transpose()
            animg = Image.fromarray(anisino * 255)
            animation_images.append(animg)

    if is_animated:
        animation_images[0].save('sinogram.gif', save_all=True, append_images=animation_images[1:])

    sinogram = np.array(pixels).transpose()

    if use_normalize:
        sinogram = normalize(sinogram)

    if use_filtering:
        sinogram = fitler(sinogram)

    return sinogram


def reversed_radon_transform(original, sinogram, alpha, n, l, is_animated):
    size_x = original.shape[1]
    size_y = original.shape[0]
    center_x = int(size_x / 2)
    center_y = int(size_y / 2)
    d = (size_x ** 2 + size_y ** 2) ** 0.5
    r = d / 2

    image_out = np.zeros((size_y, size_x))  # [[0 for _ in range(size_x)] for _ in range(size_y)]
    focals = np.zeros((size_y, size_x))
    iterations = int(360. / alpha)
    angles = np.linspace(0., 360., iterations, endpoint=False)

    animation_images = []

    for i in range(iterations):
        emiter = find_emiter_position(angles[i], r, center_x, center_y)
        detectors = find_detector_positions(n, angles[i], r, l, center_x, center_y)
        column = sinogram[:, i]

        for id, detector in enumerate(detectors):
            line = bresenham(emiter[0], emiter[1], detector[0], detector[1])
            for pixel in line:
                px, py = pixel[0], pixel[1]
                if (px >= 0) and (py >= 0) and (px < size_x) and (py < size_y):
                    image_out[py][px] += column[id]
                    focals[py][px] += 1

        if is_animated:
            # jeśli jest dużo iteracji, zapisz co trzecią klatkę animacji
            if iterations >= 60 and i % 3 != 0:
                continue
            aniout = image_out.copy()
            aniout = np.array(normalize(aniout))
            animg = Image.fromarray(aniout * 255)
            animation_images.append(animg)

    if is_animated:
        animation_images[0].save('image_out.gif', save_all=True, append_images=animation_images[1:])

    normalize(image_out)
    return image_out


def normalize(img):
    max_value = np.max(img)
    min_value = np.min(img)
    if max_value == min_value:
        return img
    normalized_img = (img - min_value) / (max_value - min_value)
    return normalized_img 


def fitler(img):
    size = 11
    mask = []

    for k in range(-size // 2, size // 2):
        if k == 0:
            mask.append(1)
        else:
            if k % 2 == 0:
                mask.append(0)
            else:
                mask.append((-4 / (np.pi * np.pi)) / (k * k))

    for i in range(img.shape[0]):
        img[i, :] = np.convolve(img[i, :], mask, 'same')

    return img


def calculate_error(original, image_out):
    # returns value of root-mean-square error (RMSE) between original and final images
    original = normalize(original)
    image_out = normalize(image_out)
    return ((np.square(original - image_out)).mean()) ** 0.5


def write_dicom(img, filename, first_name, last_name, patient_id,
                sex, birth_date, study_date, comments):
    img = img_as_int(img)
    filect = get_testdata_file('CT_small.dcm')
    ds = pydicom.dcmread(filect)

    ds.PatientName = f"{last_name}^{first_name}"
    ds.PatientID = patient_id
    ds.PatientSex = sex
    ds.PatientBirthDate = birth_date
    ds.StudyDate = study_date

    ds.Rows, ds.Columns = img.shape[0], img.shape[1]
    ds.PixelRepresentation = 0
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15
    ds.PixelData = img.tobytes()
    ds.ImageComments = comments

    ds.save_as(filename)


def main(image, alpha, n, l, use_filtering, use_normalize, is_animated):
    print("Generating sinogram...")
    sino = radon_transform(image, alpha, n, l, use_filtering, use_normalize, is_animated)
    
    if is_animated:
        s = open('sinogram.gif', 'rb')
        s = s.read()
        sani = widgets.Image(value=s, width=250)
        print("Animated sinogram")
        display(sani)
    
    print("Generating output image...")
    image_out = normalize(reversed_radon_transform(image, sino, alpha, n, l, is_animated))
    
    if is_animated:
        o = open('image_out.gif', 'rb')
        o = o.read()
        oani = widgets.Image(value=o, width=250)
        print("Animated output image")
        display(oani)
    
    print("Process finished! Preparing results...")

    fig = plt.figure(figsize=(14, 4), dpi=100)

    sub1 = fig.add_subplot(131)
    sub1.set_title("Original image")
    sub1.set_xlabel("x")
    sub1.set_ylabel("y")
    sub1.imshow(image, cmap='gray')

    sub2 = fig.add_subplot(132)
    sub2.set_title("Sinogram")
    sub2.set_xlabel("iterations")
    sub2.set_ylabel("detectors")
    sub2.imshow(sino, cmap='gray')

    sub3 = fig.add_subplot(133)
    sub3.set_title("Output image")
    sub3.set_xlabel("x")
    sub3.set_ylabel("y")
    sub3.imshow(image_out, cmap='gray')

    plt.show()
        
    return image_out

In [3]:
def run_app_clicked(b):
    clear_output(wait=True)
    create_gui()
    
    uploaded = False
    
    for filename in upload_image.value:
        content = upload_image.value[filename]['content']
        image = Image.open(io.BytesIO(content))
        image = ImageOps.grayscale(image)
        image = np.asarray(image)
        uploaded = True
        
    if not uploaded:
        print("Upload an image before ruuning the app!")
        return
    
    alpha = alpha_slider.value
    n = n_slider.value
    l = l_slider.value
    use_filtering = fitler_check.value
    use_normalize = normalize_check.value
    is_animated = anime_check.value
    show_error = error_check.value

    image_out = main(image, alpha, n, l, use_filtering, use_normalize, is_animated)
    
    if show_error:
        print("Root-mean-square error: ", calculate_error(image, image_out))
    

    def save_dicom_clicked(b):
        write_dicom(image_out,
                   filename_text.value,
                   firstname_text.value,
                   lastname_text.value,
                   patientID_text.value,
                   sext.value,
                   birthday_text.value,
                   studydate_text.value,
                   comments_text.value)
        print("DICOM file has been created.")
    
    save_dicom_button = widgets.Button(description='Save as DICOM')
    save_dicom_button.on_click(save_dicom_clicked)
    
    def show_dicom_clicked(b):
        display(dicom_grid)
        display(save_dicom_button)
    
    show_dicom_button = widgets.Button(description='Create DICOM')
    show_dicom_button.on_click(show_dicom_clicked)
    display(show_dicom_button)
    
    
    
# ------------------------------------------------------------------------------------------------------

def create_gui():
    display(upload_image)
    display(selection_grid)
    display(fitler_check, anime_check, error_check)
    display(run_app_button)


# ------------------------------------------------------------------------------------------------------
    
upload_image = widgets.FileUpload(accept='image/*', multiple=False, description='Load image')

run_app_button = widgets.Button(description='Run')
run_app_button.on_click(run_app_clicked)

alpha_slider = widgets.FloatSlider(min=0.5, max=60.0, value=3.0, step=0.5)  # layout's step
n_slider = widgets.IntSlider(min=10, max=750, value=120, step=10)  # detectors
l_slider = widgets.IntSlider(min=30, max=270, value=120, step=5)  # plane angle

sliders = [widgets.Label(value="Emiter/detector layout's angle step (alpha):"), alpha_slider,
           widgets.Label(value="Number of detectors (n):"), n_slider,
           widgets.Label(value="Plane angle of emiter/detector layout (l):"), l_slider]

selection_grid = widgets.GridBox(sliders, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))

normalize_check = widgets.Checkbox(value=True, description='Use image normalization:')
fitler_check = widgets.Checkbox(value=False, description='Use image filtering:')
anime_check = widgets.Checkbox(value=False, description='Animate results:')
error_check = widgets.Checkbox(value=True, description='Show RMSE:')


# ------------------------------------------------------------------------------------------------------

filename_text = widgets.Text()
firstname_text = widgets.Text()
lastname_text = widgets.Text()
patientID_text = widgets.Text()
sext = widgets.Text()
birthday_text = widgets.Text()
studydate_text = widgets.Text()
comments_text = widgets.Text()

dicom_texts = [widgets.Label(value="Filename (.dcm):"), filename_text,
              widgets.Label(value="First name:"), firstname_text,
              widgets.Label(value="Last name:"), lastname_text,
              widgets.Label(value="Patient ID:"), patientID_text,
              widgets.Label(value="Sex:"), sext,
              widgets.Label(value="Date of birth:"), birthday_text,
              widgets.Label(value="Date of study:"), studydate_text,
              widgets.Label(value="Comments:"), comments_text]

dicom_grid = widgets.GridBox(dicom_texts, layout=widgets.Layout(grid_template_columns="repeat(2, 150px)"))

create_gui()


FileUpload(value={}, accept='image/*', description='Load image')

GridBox(children=(Label(value="Emiter/detector layout's angle step (alpha):"), FloatSlider(value=3.0, max=60.0…

Checkbox(value=False, description='Use image filtering:')

Checkbox(value=False, description='Animate results:')

Checkbox(value=True, description='Show RMSE:')

Button(description='Run', style=ButtonStyle())

## DICOM file reader

In [4]:
def read_dicom_clicked(b):
    clear_output(wait=True)
    display(dicom_button_grid)
    uploaded = False
    success = False
    
    for filename in upload_dicom.value:
        content = upload_dicom.value[filename]['content']
        data = DicomBytesIO(content)
        dt = pydicom.dcmread(data)
        uploaded = True
        
    if not uploaded:
        print("Upload DICOM file!")
        return
    
    for info in dt:
        if info.keyword == 'PatientID':
            print(f"Patient ID: {dt.PatientID}")
        elif info.keyword == 'PatientName':
            print(f"Name: {dt.PatientName.given_name} {dt.PatientName.family_name}")
        elif info.keyword == 'PatientSex':
            print(f"Sex: {dt.PatientSex}")
        elif info.keyword == 'PatientBirthDate':
            print(f"Date of birth: {dt.PatientBirthDate}")
        elif info.keyword == 'StudyDate':
            print(f"Date of study: {dt.StudyDate}")
        elif info.keyword == 'ImageComments':
            print(f"Comments: {dt.ImageComments}")
        elif info.keyword == 'PixelData':
            success = True
    
    if success:
        plt.imshow(dt.pixel_array, cmap='gray')
        
# ------------------------------------------------------------------------------------------------------

upload_dicom = widgets.FileUpload(accept=".dcm", multiple=False, description="Upload file")

dicom_read_button = widgets.Button(description="Read DICOM")  
dicom_read_button.on_click(read_dicom_clicked)

dicom_buttons = [widgets.Label(value="Upload DICOM file:"), upload_dicom,
                 widgets.Label(value="Read DICOM file:"), dicom_read_button]

dicom_button_grid = widgets.GridBox(dicom_buttons, layout=widgets.Layout(grid_template_columns="repeat(2, 250px)"))


display(dicom_button_grid)

GridBox(children=(Label(value='Upload DICOM file:'), FileUpload(value={}, accept='.dcm', description='Upload f…